# Various working examples of autoencoders

Ref: [Keras blog](https://blog.keras.io/building-autoencoders-in-keras.html)

In [4]:
from keras.layers import Input, Dense
from keras.models import Model, Sequential

In [8]:
#the size of the encoded representation
encodingDim = 32 # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

In [7]:
# placeholder for the input image
inputImg = Input(shape=(784,))

"encoded" is the encoded representation of the input

In [9]:
encoded = Dense(encodingDim, activation='relu')(inputImg)

In [10]:
decoded = Dense(784, activation='sigmoid')(encoded)

In [11]:
autoencoder = Model(inputImg, decoded)

In [12]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                25120     
_________________________________________________________________
dense_2 (Dense)              (None, 784)               25872     
Total params: 50,992
Trainable params: 50,992
Non-trainable params: 0
_________________________________________________________________


In [17]:
785 * 32

25120

In [18]:
33 * 784

25872

In [14]:
type(autoencoder)

keras.engine.training.Model

Let's build the same network using the Sequential class of keras. 

In [21]:
autoencoderSeq = Sequential()

In [22]:
autoencoderSeq.add(Dense(encodingDim, input_shape=(784,), activation='relu'))

In [23]:
autoencoderSeq.add(Dense(784, activation='sigmoid'))

In [25]:
autoencoderSeq.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 32)                25120     
_________________________________________________________________
dense_7 (Dense)              (None, 784)               25872     
Total params: 50,992
Trainable params: 50,992
Non-trainable params: 0
_________________________________________________________________


In [26]:
type(autoencoderSeq)

keras.models.Sequential